In [ ]:
from flask import Flask, request, render_template, jsonify
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import os
import tensorflow as tf
import base64

app = Flask(__name__)

# Load your model
model = load_model('RailwayHygieneAI.h5')

# Define your unique classes list (make sure it matches your model's output)
unique_classes_list = [
    'Indian Dustbin Bad Cleanliness',
    'Indian Dustbin Good Cleanliness',
    'Indian Dustbin Medium Cleanliness',
    'Indian Toilet Bad Cleanliness',
    'Indian Toilet Good Cleanliness',
    'Indian Toilet Medium Cleanliness',
    'Indian Washbasin Bad Cleanliness',
    'Indian Washbasin Good Cleanliness',
    'Indian Washbasin Medium Cleanliness',
    'Western Dustbin Bad Cleanliness',
    'Western Dustbin Good Cleanliness',
    'Western Dustbin Medium Cleanliness',
    'Western Toilet Bad Cleanliness',
    'Western Toilet Good Cleanliness',
    'Western Toilet Medium Cleanliness',
    'Western Washbasin Bad Cleanliness',
    'Western Washbasin Good Cleanliness',
    'Western Washbasin Medium Cleanliness'
]

def model_predict(img_path, model):
    print(f"Predicting image at path: {img_path}")
    img = image.load_img(img_path, target_size=(256, 256))  # Load and resize image
    x = image.img_to_array(img)  # Convert image to array
    x = np.expand_dims(x, axis=0)  # Expand dimensions to match batch size
    x /= 255.0  # Normalize image
    preds = model.predict(x)  # Make prediction
    predicted_class = np.argmax(preds, axis=1)[0]  # Get predicted class index
    predicted_label = unique_classes_list[predicted_class]  # Get corresponding label
    confidence = float(preds[0][predicted_class])  # Confidence score
    print(f"Prediction complete: {predicted_label} (Confidence: {confidence:.4f})")
    return predicted_label, confidence

@app.route('/', methods=['GET'])
def index():
    print("Rendering index page.")
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def upload():
    print("File upload endpoint called.")
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'})

    files = request.files.getlist('file')
    if len(files) == 0:
        return jsonify({'error': 'No files selected'})

    results = []
    for file in files:
        if file.filename == '':
            continue
        basepath = os.getcwd()
        file_path = os.path.join(basepath, 'uploads', file.filename)
        file.save(file_path)

        # Make prediction
        predicted_label, confidence = model_predict(file_path, model)

        # Encode image to base64
        with open(file_path, "rb") as img_file:
            encoded_img = base64.b64encode(img_file.read()).decode('utf-8')

        # Remove the file after prediction
        os.remove(file_path)

        results.append({'filename': file.filename, 'prediction': predicted_label, 'confidence': 100 * confidence, 'image': encoded_img})

    return jsonify({'results': results})

if __name__ == '__main__':
    print("Starting Flask app...")
    app.run(debug=True, use_reloader=False)


Starting Flask app...
 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [12/Jul/2024 14:14:08] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Jul/2024 14:14:08] "GET /static/styles.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [12/Jul/2024 14:14:08] "GET /static/scripts.js HTTP/1.1" 200 -


Rendering index page.
File upload endpoint called.
Predicting image at path: C:\Users\godof\OneDrive\Desktop\cris project\uploads\AKASHAYTHAKUR_130_12107_193124_I_D_G.jpg


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


INFO:werkzeug:127.0.0.1 - - [12/Jul/2024 14:14:16] "POST /predict HTTP/1.1" 200 -


Prediction complete: Indian Dustbin Good Cleanliness (Confidence: 0.9992)
